# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [3]:
import math, subprocess
import os
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [4]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();


df_grafos_propios = pd.read_csv("instancias/grafo-propio/indice.csv");
df_grafos_opt = pd.read_csv("instancias/descargas/indice.csv");
df_peor_caso_AGM = pd.read_csv("instancias/peor-caso-agm/indice.csv");



La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
Metodos:
- agm
- ins
- GA
- TABU_CICLOS
- TABU_ESTRUCTURA


In [5]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    display(metodo)
    #print(instancia)
    
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../TSP", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    res = process.stdout.read().split(" ")
    tam = res[0] #tamaño del ciclo
    pyc =res[1].split("\n")
    peso = pyc[0] #peso del ciclo
    ciclo = pyc[1] #ciclo

    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return [tiempo_de_ejecucion, tam, peso, ciclo]; #devuelvo un par de cosas extra por si las queremos usar

# Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

### Importante la celda de abajo para generar los experimentos!!

In [7]:
experimentos = [];
#algoritmos = ["ins", "agm", "GA",  "TABU_CICLOS", "TABU_ESTRUCTURA"]
algoritmos =["ins", "agm", "GA",  "TABU_CICLOS", "TABU_ESTRUCTURA"]; #de prueba para ver que funciona
algoritmos_tabu = ["TABU_CICLOS", "TABU_ESTRUCTURA"] # Para los experimentos de parametros

### Armado de experimentos grafos random

In [1]:

for n in range(0, df_grafos_propios.shape[0]):
    fila_n = df_grafos_propios.iloc[n];
    for algo in algoritmos:
        experimentos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);
 

NameError: name 'df_grafos_propios' is not defined

# Ejecucion de algoritmos con Grafos Random

In [ ]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo ]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado.csv", index=False, header=True);

'Experimento: 494/500'

'TABU_CICLOS'

'TABU_CICLOS'

'TABU_CICLOS'

# Ejecucion de los experimentos con optimos conocidos

In [18]:
df_grafos_opt.head()

,dataset,instancia,n,m,archivo,peso_optimo
0,sol_optimas,u1432,1432,1024596.0,instancias/descargas/data/u1432.tsp,152970
1,sol_optimas,gr666,666,221445.0,instancias/descargas/data/gr666.tsp,294358
2,sol_optimas,rl1304,1304,849556.0,instancias/descargas/data/rl1304.tsp,252948
3,sol_optimas,pr107,107,5671.0,instancias/descargas/data/pr107.tsp,44303
4,sol_optimas,rat99,99,4851.0,instancias/descargas/data/rat99.tsp,1211


In [49]:
experimentos_optimos = []


for n in range(0, df_grafos_opt.shape[0]):
    fila_n = df_grafos_opt.iloc[n];
    for algo in algoritmos:
        experimentos_optimos.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"], fila_n["peso_optimo"]]);

In [ ]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "gap"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_optimos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_optimos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    gap = costo - experimento[5] # Lo que me falto para el costo optimo conocido  
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, gap]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_opts.csv", index=False, header=True);

# Ejecucion del experimento de peor caso AGM

In [ ]:
experimentos_AGM = []


for n in range(0, df_peor_caso_AGM.shape[0]):
    fila_n = df_peor_caso_AGM.iloc[n];
    for algo in algoritmos:
        experimentos_AGM.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo, fila_n["archivo"]]);

In [ ]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_AGM:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_AGM)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        algoritmo = experimento[4]
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_agm.csv", index=False, header=True);

# Experimentacion de parametros TABU

## Con los grafos randoms

In [11]:
experimentos_TABU_random = []

for n in range(0, df_grafos_propios.shape[0]):
    fila_n = df_grafos_propios.iloc[n];
    for algo in algoritmos_tabu:
# como correr_experimentos le pasa de parametro a ./TSP TABU_CICLOS, agregamos aca como strings
# t memoria, vecindad, iteraciones
        for t in range(10,101,10):
            for v in range(10,101,10):
                for it in range(100,401,50):
                    algo_params = " " + str(t) + " " + str(v) + " " + str(it)
                    experimentos_TABU_random.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo + algo_params, fila_n["archivo"], str(t), str(v), str(it)]);

In [ ]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "memoria", "vecindad", "iteraciones"];
filas = [];
numero = 1
T = 3 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_random:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_random)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    t = experimento[5]
    v = experimento[6]
    it = experimento[7]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, t, v, it]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_params_random.csv", index=False, header=True);

## Con los grafos de opt

In [ ]:
experimentos_TABU_opt = []

for n in range(0, df_grafos_opts.shape[0]):
    fila_n = df_grafos_opts.iloc[n];
    if (fila_n["n"] < 400):
        for algo in algoritmos_tabu:
# como correr_experimentos le pasa de parametro a ./TSP TABU_CICLOS, agregamos aca como strings
# t memoria, vecindad, iteraciones
            for t in range(10,101,20):
                for v in range(10,101,20):
                    for it in range(100,401,100):
                        algo_params = " " + str(t) + " " + str(v) + " " + str(it)
                        experimentos_TABU_opt.append([fila_n["dataset"], fila_n["n"], fila_n["m"], algo + algo_params, fila_n["archivo"], str(t), str(v), str(it)]);

In [ ]:
columnas = ["dataset", "n", "m", "metodo", "tiempo", "costo", "memoria", "vecindad", "iteraciones"];
filas = [];
numero = 1
T = 3 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos_TABU_opt:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos_TABU_opt)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    costos  = []
    tiempos = []
    for i in range(0, T):
        resultado = correr_experimento(experimento[3], experimento[4])
        costos.append(resultado[2])
        tiempos.append(resultado[0]);
    costo = min(costos)
    tiempo = np.mean(tiempos);
    t = experimento[5]
    v = experimento[6]
    it = experimento[7]
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo,costo, t, v, it]);
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado.to_csv("resultados/resultado_tabu_params_opt.csv", index=False, header=True);